In [3]:
import zipfile
import os

zip_path = "archive (37).zip"
extract_path = "mrl_eye_dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Dataset extracted to:", extract_path)


✅ Dataset extracted to: mrl_eye_dataset


In [7]:
import os

dataset_dir = "mrl_eye_dataset/train"  # or the correct unzipped path
closed_dir = os.path.join(dataset_dir, "Closed_Eyes")
open_dir = os.path.join(dataset_dir, "Open_Eyes")


In [8]:
print(os.listdir("mrl_eye_dataset/train"))


['Closed_Eyes', 'Open_Eyes']


In [9]:
import os

# Try this first
print(os.listdir("mrl_eye_dataset"))


['train']


In [10]:
import os

dataset_dir = "mrl_eye_dataset/train"  # ✅ Correct path
closed_dir = os.path.join(dataset_dir, "Closed_Eyes")
open_dir = os.path.join(dataset_dir, "Open_Eyes")

print("Closed dir exists:", os.path.exists(closed_dir))  # Should return True
print("Open dir exists:", os.path.exists(open_dir))      # Should return True


Closed dir exists: True
Open dir exists: True


In [11]:
import cv2
import numpy as np

def load_images_labels(closed_dir, open_dir):
    data = []
    labels = []

    for file in os.listdir(closed_dir):
        if file.endswith(".png") or file.endswith(".jpg"):
            img = cv2.imread(os.path.join(closed_dir, file), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (24, 24))
            data.append(img)
            labels.append(0)

    for file in os.listdir(open_dir):
        if file.endswith(".png") or file.endswith(".jpg"):
            img = cv2.imread(os.path.join(open_dir, file), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (24, 24))
            data.append(img)
            labels.append(1)

    return np.array(data), np.array(labels)

# Load the data
X, y = load_images_labels(closed_dir, open_dir)
print("✅ Loaded images:", X.shape)
print("✅ Loaded labels:", y.shape)


✅ Loaded images: (4000, 24, 24)
✅ Loaded labels: (4000,)


In [12]:
# Normalize pixel values
X = X / 255.0

# Reshape to (samples, height, width, channels)
X = X.reshape(-1, 24, 24, 1)

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (3200, 24, 24, 1)
Test shape: (800, 24, 24, 1)


In [14]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp312-cp312-win_amd64.whl (376.0 MB)


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(24, 24, 1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


C:\Users\Sreeja Mondal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 22, 22, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 11, 11, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 9, 9, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 150,145 (586.50 KB)

 Trainable params: 150,145 (586.50 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - accuracy: 0.7557 - loss: 0.5043 - val_accuracy: 0.9762 - val_loss: 0.1198
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9559 - loss: 0.1378 - val_accuracy: 0.9825 - val_loss: 0.0545
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.9730 - loss: 0.0840 - val_accuracy: 0.9862 - val_loss: 0.0440
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9860 - loss: 0.0436 - val_accuracy: 0.9912 - val_loss: 0.0242
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - accuracy: 0.9885 - loss: 0.0329 - val_accuracy: 0.9950 - val_loss: 0.0150
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9941 - loss: 0.0211 - val_accuracy: 0.9925 - val_loss: 0.0212
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9979 - loss: 0.0145 - val_accuracy: 0.9950 - val_loss: 0.0132
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9945 - loss: 0.0187 - val_accu

In [17]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ Test Accuracy: {accuracy:.2f}")

# Save model
model.save("eye_state_model.h5")


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9991 - loss: 0.0022


✅ Test Accuracy: 1.00


In [18]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load trained model
model = load_model("eye_state_model.h5")


In [19]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale and resize
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (24, 24))
    normalized = resized / 255.0
    reshaped = normalized.reshape(1, 24, 24, 1)

    # Predict eye state
    prediction = model.predict(reshaped)[0][0]
    label = "Open" if prediction > 0.5 else "Closed"
    color = (0, 255, 0) if prediction > 0.5 else (0, 0, 255)

    # Display label on frame
    cv2.putText(frame, f"Eye: {label}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    cv2.imshow("Real-time Eye Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━